# GISCUP 2023 Solution

Authors: Adam Booth, Richard Burke

Contact: a.booth2@newcastle.ac.uk, r.burke1@newcastle.ac.uk

This notebook provides the solution we developed as a team for the GISCUP 2023 competition.

The notebook extracts train/test regions from the provided Sentinel-2 imagergy and proceeds to generate lake delineated bitmasks on the test regions only.

Once the bitmasks are generated, a post-processing step is taken, to filter out lakes which do not meet the requirements and generates a shapefile containing the detected lakes.

The model imported in this notebook (RGB_256_filters_4) is a Tensorflow/Keras model, trained on the provided training regions and can be found in the Model_Training notebook.

In [ ]:
from pathlib import Path
import tensorflow as tf
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.features import rasterize
import math
from rasterio.windows import Window
import numpy as np
from shapely.geometry import Polygon
from functools import reduce
from osgeo import gdal, ogr, osr
from shapely.geometry import LineString

## Data Extraction

In [ ]:
# Create folders to work in
ROOT_DIR = Path("../data/")
DATA_PATH = Path("../data/2023_SIGSPATIAL_Cup_data_files/")
TEMP_DIR = ROOT_DIR/"temp"
if not TEMP_DIR.exists():
    TEMP_DIR.mkdir()

TRAIN_REGIONS_EXT_DIR = TEMP_DIR/"regions/train"
TEST_REGIONS_EXT_DIR = TEMP_DIR/"regions/test"

if not TRAIN_REGIONS_EXT_DIR.exists():
    TRAIN_REGIONS_EXT_DIR.mkdir(parents=True)

if not TEST_REGIONS_EXT_DIR.exists():
    TEST_REGIONS_EXT_DIR.mkdir(parents=True)

TEST_REGIONS_BMASK_DIR = ROOT_DIR/"lakes_bitmask/test_regions"
if not TEST_REGIONS_BMASK_DIR.exists():
    TEST_REGIONS_BMASK_DIR.mkdir(parents=True)

POST_PROCESSING_DIR = Path("../data/post_processing")
if not POST_PROCESSING_DIR.exists():
    POST_PROCESSING_DIR.mkdir(parents=True)


In [ ]:
# Load regions and sentinal data
regions = gpd.read_file(DATA_PATH/"lakes_regions.gpkg")
sentinal_files = list(Path(DATA_PATH).glob("*.tif"))

In [ ]:
train_regions = ["2019-06-03_2", "2019-06-03_4", "2019-06-03_6",
                 "2019-06-19_1", "2019-06-19_3", "2019-06-19_5",
                 "2019-07-31_2", "2019-07-31_4", "2019-07-31_6",
                 "2019-08-25_1", "2019-08-25_3", "2019-08-25_5"]

test_regions = ["2019-06-03_1", "2019-06-03_3", "2019-06-03_5",
                "2019-06-19_2", "2019-06-19_4", "2019-06-19_6",
                "2019-07-31_1", "2019-07-31_3", "2019-07-31_5",
                 "2019-08-25_2", "2019-08-25_4", "2019-08-25_6" ]

In [ ]:
# Extract regions
for _, region in regions.iterrows():
    for sentinal_file in sentinal_files:
        region_num = region["region_num"]
        date = str(sentinal_file)[-17: -7]
        
        with rasterio.open(sentinal_file) as sentinal_image:
            region_raw, affine = mask(sentinal_image, shapes=[region.geometry], crop=True)
            
        file_name = f"{date}_{region_num}"

        out_dir = TEST_REGIONS_EXT_DIR if file_name in test_regions else TRAIN_REGIONS_EXT_DIR

        with rasterio.open(out_dir/f"{file_name}.tif", 
                           "w", driver=sentinal_image.driver, crs=sentinal_image.crs, 
                           transform=affine, height=region_raw.shape[1], width=region_raw.shape[2], 
                           count=3, dtype=region_raw.dtype) as out:
            out.write(region_raw)

## Lake Delineation using DUCK-Net Model (see 02_model_training.ipynb)

In [ ]:
# Load pre-trained model
model = tf.keras.models.load_model("../model/RGB_256_filters_4/", compile=False)

# Generate bitmasks on test regions, using pre-trained model
for region_file in TEST_REGIONS_EXT_DIR.glob("*.tif"):

    with rasterio.open(region_file) as region_sat:
        date, region = (region_file.stem[:10], region_file.stem[11:])
        WINDOW_SIZE = 256

        num_cols = math.ceil(region_sat.width / WINDOW_SIZE)
        num_rows = math.ceil(region_sat.height / WINDOW_SIZE)
        window_indices = [(col, row) for col in range(num_cols) for row in range(num_rows)]

        out_bitmask = np.zeros((region_sat.height, region_sat.width, 1), dtype=np.uint8)

        for col_indx, row_indx in window_indices:
            # Read window
            window = Window(col_indx * WINDOW_SIZE, row_indx * WINDOW_SIZE, WINDOW_SIZE, WINDOW_SIZE)
            window_read = region_sat.read(window=window)

            # Reshape and sacle for inference
            window_read = np.moveaxis(window_read, 0, 2)
            window_read = np.expand_dims(window_read, axis=0)
            window_read = window_read / 255

            if window_read.shape[1] != WINDOW_SIZE or window_read.shape[2] != WINDOW_SIZE:
                continue

            # Predict bitmask for slice
            pred = model.predict(window_read, verbose=0)
            window_bitmask = pred[0]
            window_bitmask = np.round(window_bitmask).astype(np.uint8)

            out_bitmask[row_indx*WINDOW_SIZE:row_indx*WINDOW_SIZE+WINDOW_SIZE, col_indx*WINDOW_SIZE:col_indx*WINDOW_SIZE+WINDOW_SIZE] = window_bitmask

        with rasterio.open(TEST_REGIONS_BMASK_DIR/f"{date}_{region}.tif", "w", driver=region_sat.driver, crs=region_sat.crs, transform=region_sat.transform, width=region_sat.shape[1], height=region_sat.shape[0], count=1, dtype=out_bitmask.dtype) as out:
            out.write(out_bitmask.squeeze(), indexes=1)

    

# Post Processing

In [ ]:
# Set the working directory

for tif_file in TEST_REGIONS_BMASK_DIR.glob("*.tif"):

    # Open the Raster File
    catchment_raster = gdal.Open(str(tif_file))

 

    # Read the Raster Band

    band = catchment_raster.GetRasterBand(1)

    band.ReadAsArray()

 

    # Get the Projection

    proj = catchment_raster.GetProjection()

    shp_proj = osr.SpatialReference()

    shp_proj.ImportFromWkt(proj)

 

    # Set Up the Output Shapefile

    output_file = POST_PROCESSING_DIR / f"{tif_file.stem}.shp"

    call_drive = ogr.GetDriverByName("ESRI Shapefile")

    if output_file.exists():
        call_drive.DeleteDataSource(str(output_file))

    create_shp = call_drive.CreateDataSource(str(output_file))

    shp_layer = create_shp.CreateLayer("layername", srs=shp_proj)

 

    # Add a Field to the Shapefile

    new_field = ogr.FieldDefn(str('value'), ogr.OFTInteger)

    shp_layer.CreateField(new_field)

 

    # Convert Raster to training_vector

    gdal.Polygonize(band, None, shp_layer, 0, [], callback=None)

 

    # Cleanup

    shp_layer = None

    create_shp = None

    raster = None

 

print("All rasters have been converted to shapefiles.")

 

#Merge shapefiles together and keep filename in row
gdfs = []

 

for shp_file in POST_PROCESSING_DIR.glob("*.shp"):

    # Read the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shp_file)
 

    # Add a new column with the filename (without extension)
    gdf['filename'] = shp_file.stem


    # Append the GeoDataFrame to the list

    gdfs.append(gdf)

 

# Concatenate all GeoDataFrames into a single GeoDataFrame

merged_gdf = gpd.pd.concat(gdfs, ignore_index=True)

 

#Create 'region_num' column

merged_gdf['region_num'] = merged_gdf['filename'].str[-1]

 

#Modify 'filename' column

merged_gdf['filename'] = 'Greenland26X_22W_Sentinel2_' + merged_gdf['filename'].str[:-2]

 

#Append specified suffixes based on the ending of 'filename'

conditions = [

    merged_gdf['filename'].str.endswith('2019-06-03'),

    merged_gdf['filename'].str.endswith('2019-06-19'),

    merged_gdf['filename'].str.endswith('2019-07-31'),

    merged_gdf['filename'].str.endswith('2019-08-25')

]

choices = ['_05.tif', '_20.tif', '_25.tif', '_29.tif']

 

merged_gdf['filename'] += np.select(conditions, choices, default='')

 

#Rename 'filename' column to 'image'

merged_gdf = merged_gdf.rename(columns={'filename': 'image'})

 

# Display the modified GeoDataFrame

print(merged_gdf)

 

#Tidy up merged_gdf.

merged_gdf = merged_gdf[merged_gdf['value'] != 0] #Remove zero value rows

merged_gdf = gpd.clip(merged_gdf, regions) #Clip to

merged_gdf = merged_gdf.explode() #Explode the MultiPolygons into individual Polygons

 

#Fill holes in merged_gdf

sizelim = 100000000000 #Fill holes less than x m2

def fillit(row):

    newgeom=None

    rings = [i for i in row["geometry"].interiors] #List all interior rings

    if len(rings)>0: #If there are any rings

        to_fill = [Polygon(ring) for ring in rings if Polygon(ring).area<sizelim] #List the ones to fill

        if len(to_fill)>0: #If there are any to fill

            newgeom = reduce(lambda geom1, geom2: geom1.union(geom2),[row["geometry"]]+to_fill) #Union the original geometry with all holes

    if newgeom:

        return newgeom

    else:

        return row["geometry"]

 

#Remove long and narrow polygons

def is_lake(polygon):

    # Compute the minimum rotated rectangle

    min_rotated_rect = polygon.minimum_rotated_rectangle

 

    # Calculate width and height

    coords = list(min_rotated_rect.exterior.coords)

    width = ((coords[0][0] - coords[1][0]) ** 2 + (coords[0][1] - coords[1][1]) ** 2) ** 0.5

    height = ((coords[1][0] - coords[2][0]) ** 2 + (coords[1][1] - coords[2][1]) ** 2) ** 0.5

 

    # Calculate the ratio

    ratio = min(width, height) / max(width, height)

 

    # Return whether the polygon is a lake

    return ratio >= 0.1

 

# Apply the functions to each row and filter the GeoDataFrame

merged_gdf["geometry"] = merged_gdf.apply(fillit, axis=1)

merged_gdf['is_lake'] = merged_gdf['geometry'].apply(is_lake)

merged_gdf = merged_gdf[merged_gdf['is_lake']]

 

#Calculate area for the filled polygons

merged_gdf['area'] = merged_gdf.geometry.area

merged_gdf['area'] = merged_gdf['area'].astype(int)

 

#Now remove any rows below 100000m2

merged_gdf = merged_gdf[(merged_gdf['area'] > 100000)]

 

merged_gdf = merged_gdf.drop(columns=['area', 'is_lake', 'value'])

 

#Export to geopackage

merged_gdf.to_file("../data/lake_polygons_test.gpkg", layer='lakes', driver="GPKG", index=False)